In [10]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.iolib.summary2 as summary2
import logging

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/JTH_reg_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-10-26 18:53:04.925460


In [12]:
pathdir = "DataStore/2018-10/Reg1016/"
reg_f = "reg_model_1016.pkl"

regs = pickle.load(open(pathdir+reg_f, "rb"))

In [13]:
regs.keys()

dict_keys(['model_names', 'n_eqns', 'ldavecs', 'docvecs', 'tp_pct_common_cited', 'JTH_model_names', 'JTH_cite'])

In [14]:

info_dict = {'$N$':lambda x: "{0:d}".format(int(x.nobs)),
'Adjusted $R^2$':lambda x: "{:.2f}".format(x.rsquared_adj)}
def get_fit(formula, grouped_data, group_col, cov_type = "HC1", return_fit = False):
    summ = []
    tables = {}
    
    # If formula uses mean similarity, use grant year above 1980
    if "mean_sim_" in formula:
        grouped_data = grouped_data.loc[(grouped_data["tp_gyear"] >= 1980)]
    
    # Remove missing values used in formula
    col_used = re.findall('\((.*?)\)',formula)
    # Intersect with grouped_data columns
    col_used = list(set(col_used).intersection(set(list(grouped_data.columns))))
    print(col_used)
    
    grouped_data = grouped_data.dropna(how="any", subset=col_used).copy().reset_index(drop=True)
    
    # Get length of data
    print(("Length of data", len(grouped_data)))
                                    
    # Group and then get results
    grouped_data = grouped_data.groupby(group_col)
    
    for n,g in grouped_data:
        try:
            if cov_type == "HC1":
                fit = smf.ols(formula = formula, data = g, missing="drop").fit(cov_type="HC1")
            else:
                # Drop missing in the grouped data first
                # Cluster uses primary class so must have primary class FE
                cols_used2 = list(set(col_used).intersection(set(list(grouped_data.columns))))+["tp_primclass_FE"]
                g = g.dropna(subset=cols_used2, how="any").copy().reset_index(drop=True)
                fit = smf.ols(formula = formula, data = g).fit(cov_type="cluster",
                                                                              cov_kwds={'groups': g["tp_primclass_FE"]})
            # Get results tables
            tables[n] = fit.summary2().tables
            # Append results
            summ.append(fit)
        except Exception as e:
            print(n)
            logging.exception("Regression error")
            pass
    # Get full results output
    # Dataframe of full results
    res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict).tables[0]
    
    # Get partial results
    # 1. Get relevant variables from index of full results: UPDATED
    regressors = [v for v in res_no_stars.index.unique() if ("sim_" in v) | ("match" in v) | ("common_" in v)]
    # 2. Make sure regressors come last
    regressors = regressors+["Intercept"]
    # 3. Get results with regressors
    part_res_no_stars = summary2.summary_col(summ, stars = False, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    part_res_stars = summary2.summary_col(summ, stars = True, \
    model_names = ["{0}".format(n) for n in tables.keys()],\
        info_dict = info_dict, regressor_order = regressors).tables[0]
    
    # 4. Get index of where Intercept is and add 2 (to include standard error)
    last_ind = list(part_res_stars.index).index("Intercept")+2
    
    # 5. Get partial results
    part_res_no_stars = pd.concat([part_res_no_stars.iloc[:last_ind], part_res_no_stars.iloc[-2::]])
    part_res_stars = pd.concat([part_res_stars.iloc[:last_ind], part_res_stars.iloc[-2::]])
    
    if return_fit == True:
        return summ, tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars
    else:
        return tables, res_no_stars, res_stars, part_res_no_stars, part_res_stars
    

In [ ]:
rs = pd.read_pickle("DataStore/2018-10/Reg1016/JTH_cite_reg_1026.pkl")

print(datetime.datetime.now())
# 3. Define output
cov = "HC1"
samp_out = {}
formulas = list(regs['JTH_cite'])
formulas_ind = list(regs['JTH_cite'].index)
for i, j in zip(formulas_ind, formulas):
    print((i, j))
    print(datetime.datetime.now())
    try:
        out = get_fit(j, rs, "year_group", cov, return_fit = False)
        samp_out[i] = {}
        samp_out[i]["model"] = j
        samp_out[i]["tables"] = out[0]
        samp_out[i]["res_no_stars"] = out[1]
        samp_out[i]["res_stars"] = out[2]
        samp_out[i]["part_res_no_stars"] = out[3]
        samp_out[i]["part_res_stars"] = out[4]
    except Exception as e:
        logging.exception("error here")
        pass
    print("finished")
    print(datetime.datetime.now())

# Define outfile
o_f = "reg_JTH_HC1_out_1026.pkl"
pickle.dump(samp_out, open(pathdir+o_f, "wb")) 

INFO:root:2018-10-26 18:53:23.973025
INFO:root:(0, 'perc_match_10 ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE)')
INFO:root:2018-10-26 18:53:23.988052
INFO:root:['tp_gyear', 'tp_primclass_FE', 'inv_msa_match']
INFO:root:('Length of data', 564158)
INFO:root:finished
INFO:root:2018-10-26 18:54:02.491542
INFO:root:(1, 'perc_match_10 ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE) + C(tp_inv_msa_FE)')
INFO:root:2018-10-26 18:54:02.493719
INFO:root:['tp_gyear', 'tp_inv_msa_FE', 'tp_primclass_FE', 'inv_msa_match']
INFO:root:('Length of data', 564158)
INFO:root:finished
INFO:root:2018-10-26 18:54:54.153022
INFO:root:(2, 'perc_match_10 ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE)  + C(tp_inv_msa_FE)    + C(tp_lawyer_FE)')
INFO:root:2018-10-26 18:54:54.155540
INFO:root:['tp_gyear', 'tp_inv_msa_FE', 'tp_lawyer_FE', 'inv_msa_match', 'tp_primclass_FE']
INFO:root:('Length of data', 564158)
INFO:root:finished
INFO:root:2018-10-26 18:56:06.689626
INFO:root:(3, 'perc_match_10 ~ 

In [17]:
samp_out

{0: {'model': 'perc_match_10 ~ C(inv_msa_match) + C(tp_gyear) + C(tp_primclass_FE)',
  'tables': {'1975-85': [                     0                 1                    2            3
    0               Model:               OLS      Adj. R-squared:        0.027
    1  Dependent Variable:     perc_match_10                 AIC:  -19126.0169
    2                Date:  2018-10-26 18:53                 BIC:  -16647.7324
    3    No. Observations:             58647      Log-Likelihood:       9839.0
    4            Df Model:               275         F-statistic:        7.247
    5        Df Residuals:             58371  Prob (F-statistic):    2.21e-252
    6           R-squared:             0.031               Scale:     0.042059,
                                    Coef.  Std.Err.          z          P>|z|  \
    Intercept                    0.045635  0.021777   2.095614   3.611646e-02   
    C(inv_msa_match)[T.True]     0.052439  0.001592  32.932832  7.451187e-238   
    C(tp_gyear)[T.

In [19]:
res_out = pd.DataFrame()
res = samp_out
for k in res.keys():
    lks = res[k]["model"].split(" ~ ")[0]

    # Selecting portion of results without intercept
    cdf = res[k]["part_res_stars"].reset_index()
    ic_ind = cdf.loc[cdf["index"] == "Intercept"].index[0]
    # Include N & R^2
    cdf = cdf.iloc[pd.np.r_[0:ic_ind,ic_ind+2:len(cdf)]]
    # Just include R^2
#             cdf = cdf.iloc[pd.np.r_[0:ic_ind,ic_ind+2:len(cdf)-2, len(cdf)-1:len(cdf)]]
    cdf["Model"] = regs["JTH_model_names"][k]
    cdf["Model Num"] = k
    cdf["LKS"] = lks

    res_out = res_out.append(cdf)

In [21]:
res_out.to_csv("DataStore/2018-10/Reg1016/JTH_cite_reg_1026.csv")